In [34]:
#tạo dataframe từ dict
import pandas as pd
d = { 'x': [1,2,3], 'y':[2,4,8],'z': 100}
f = pd.DataFrame(d)
print(f)


   x  y    z
0  1  2  100
1  2  4  100
2  3  8  100


In [35]:
d = {'x': [1,2,3], 'y': [2,4,8], 'z': 100}
f = pd.DataFrame(d, index=[10,20,30], columns=['z','y','x'])
print(f)

      z  y  x
10  100  2  1
20  100  4  2
30  100  8  3


In [36]:
#tạo dataframe từ csv
df_wt = pd.read_csv('weather.csv')
print(df_wt)

     Outlook Temperature Humidity    Wind Play
0      Sunny         Hot     High    Weak   No
1      Sunny         Hot     High  Strong   No
2   Overcast         Hot     High    Weak  Yes
3       Rain        Mild     High    Weak  Yes
4       Rain        Cool   Normal    Weak  Yes
5       Rain        Cool   Normal  Strong   No
6   Overcast        Cool   Normal  Strong  Yes
7      Sunny        Mild     High    Weak   No
8      Sunny        Cool   Normal    Weak  Yes
9       Rain        Mild   Normal    Weak  Yes
10     Sunny        Mild   Normal  Strong  Yes
11  Overcast        Mild     High  Strong  Yes
12  Overcast         Hot   Normal    Weak  Yes
13      Rain        Mild     High  Strong   No


2. Cây quyết định (ID3)

In [37]:
import pandas as pd
import numpy as np
#Đọc dữ liệu từ file .csv và lưu vào dataframe
train_data = pd.read_csv('weather.csv')
print(train_data)

     Outlook Temperature Humidity    Wind Play
0      Sunny         Hot     High    Weak   No
1      Sunny         Hot     High  Strong   No
2   Overcast         Hot     High    Weak  Yes
3       Rain        Mild     High    Weak  Yes
4       Rain        Cool   Normal    Weak  Yes
5       Rain        Cool   Normal  Strong   No
6   Overcast        Cool   Normal  Strong  Yes
7      Sunny        Mild     High    Weak   No
8      Sunny        Cool   Normal    Weak  Yes
9       Rain        Mild   Normal    Weak  Yes
10     Sunny        Mild   Normal  Strong  Yes
11  Overcast        Mild     High  Strong  Yes
12  Overcast         Hot   Normal    Weak  Yes
13      Rain        Mild     High  Strong   No


In [38]:
#Xây dựng hà tính entropy
def entropy(data, class_col):
    #Nhóm các giá trị khác nhau và đếm dòng dữ liệu có giá trị tương ứng
    lb_count = data[class_col].groupby(data[class_col]).count().values
    #Tính xác suất
    p = lb_count/sum(lb_count)
    #Tính entropy
    entropy_val = sum(-p* np.log2(p))
    return entropy_val

In [39]:
#Xây dựng hàm tính infomation gain
def information_gain(data, att, class_col):
    entropy_data = entropy(data, class_col)
    entropy_part = [] #entropy của các phân vùng
    length_part = []
    #Với mỗi giá trị của cột att
    for att_val in data[att].unique():
        #Xác định phân vùng dữ liệu ứng với giá trị att_val của cột att
        partition = data[data[att] == att_val] #fix
        #Tính entropy ứng với phân vùng
        entropy_part.append(entropy(partition, class_col))
        #Lưu số dòng ứng với phân vùng
        length_part.append(len(partition))

    entropy_part = np.array(entropy_part)
    ratio_part = np.array(length_part) / len(data)
    
    ig = entropy_data - np.sum(ratio_part * entropy_part)
    return ig

In [40]:
#Kiểm tra một phân vùng có thuần nhất không
def check_purity(data, class_col):
    if len(np.unique(data[class_col])) == 1:
        return True
    else:
        return False
    

In [41]:
#Xác định nhãn của một nhánh
def classify(data, class_col):
    classes, count_classes = np.unique(data[class_col], return_counts=True)
    index = count_classes.argmax()
    return classes[index]

In [42]:
#Hàm xây dựng cây
#Cây quyết định: Định dạng là dictionary
#Mỗi phần tử là danh sách của cá dictionary
def build_decision_tree(data, class_col, visited):
    if check_purity(data, class_col):
        return {'label': classify(data, class_col)}
    
    max_ig = -1 #lưu information gain lớn nhất
    max_ig_col = '' #lưu cột (thuộc tính) ứng với gain lớn nhất

    #duyệt qua các thuộc tính
    for column in data.columns.values:
        #nếu thuộc tính đã chưa xét và không phải cột lớp
        if column not in visited and column != class_col:
            #Tính information gain của thuộc tính đó
            ig = information_gain(data, column, class_col)
            if ig > max_ig:
                max_ig = ig
                max_ig_col = column
    if max_ig <= 0:
        return {'label': classify(data, class_col)}
    
    visited.append(max_ig_col)
    sub_tree = {'node': max_ig_col, 'branches': [], 'label': None}

    for att_val in data[max_ig_col].unique():
        print(max_ig_col, att_val)
        sub_tree_att = {}
        sub_tree_att['branch'] = att_val
        sub_tree_att['next'] = build_decision_tree(data[data[max_ig_col] == att_val],
                                                        class_col, visited.copy())
        sub_tree['branches'].append(sub_tree_att)
    return sub_tree

In [43]:
#Hàm dự đoán nhãn của một mẫu dữ liệu đâu vào
def classify_example(example, decision_tree):
    if('label' in decision_tree.keys()) and (decision_tree['label'] != None):
        return decision_tree['label']
    
    question = decision_tree['node']
    for branch in decision_tree['branches']:
        if branch['branch'] == example[question]:
            return classify_example(example, branch['next'])
    return 'Không tìm thấy'

In [44]:
col_of_class = 'Play'
#Thực hiện
visited = []
tree = build_decision_tree(train_data, col_of_class, visited)
print(tree)
example = {'Outlook':'rainy', 'Temperature':'hot', 'Humidity':'normal', 'Wind':'strong'}
classify_example(example, tree)


Outlook Sunny
Humidity High
Humidity Normal
Outlook Overcast
Outlook Rain
Wind Weak
Wind Strong
{'node': 'Outlook', 'branches': [{'branch': 'Sunny', 'next': {'node': 'Humidity', 'branches': [{'branch': 'High', 'next': {'label': 'No'}}, {'branch': 'Normal', 'next': {'label': 'Yes'}}], 'label': None}}, {'branch': 'Overcast', 'next': {'label': 'Yes'}}, {'branch': 'Rain', 'next': {'node': 'Wind', 'branches': [{'branch': 'Weak', 'next': {'label': 'Yes'}}, {'branch': 'Strong', 'next': {'label': 'No'}}], 'label': None}}], 'label': None}


'Không tìm thấy'

In [47]:
def print_tree(decision_tree, indent=""):
    """
    Hàm đệ quy để in cây quyết định ra màn hình một cách trực quan.
    """
    # Nếu là nút lá (có kết quả dự đoán)
    if 'label' in decision_tree and decision_tree['label'] is not None:
        print(f"{indent}└── Predict: {decision_tree['label']}")
        return

    # Nếu là nút quyết định (câu hỏi)
    node_question = decision_tree['node']
    print(f"{indent}{node_question}?")

    # Duyệt qua các nhánh con
    for i, branch in enumerate(decision_tree['branches']):
        branch_value = branch['branch']
        # Dùng ký tự khác cho nhánh cuối cùng để đẹp hơn
        if i == len(decision_tree['branches']) - 1:
            connector = "└──"
        else:
            connector = "├──"
        
        print(f"{indent} {connector} {branch_value}:")
        # Gọi đệ quy để in cây con, tăng mức thụt lề
        print_tree(branch['next'], indent + "    |")

# Dữ liệu cây của bạn
tree = {'node': 'Outlook', 'branches': [{'branch': 'Sunny', 'next': {'node': 'Humidity', 'branches': [{'branch': 'High', 'next': {'label': 'No'}}, {'branch': 'Normal', 'next': {'label': 'Yes'}}], 'label': None}}, {'branch': 'Overcast', 'next': {'label': 'Yes'}}, {'branch': 'Rain', 'next': {'node': 'Wind', 'branches': [{'branch': 'Weak', 'next': {'label': 'Yes'}}, {'branch': 'Strong', 'next': {'label': 'No'}}], 'label': None}}], 'label': None}

# Gọi hàm để in cây
print("--- Cấu Trúc Cây Quyết Định ---")
print_tree(tree)

--- Cấu Trúc Cây Quyết Định ---
Outlook?
 ├── Sunny:
    |Humidity?
    | ├── High:
    |    |└── Predict: No
    | └── Normal:
    |    |└── Predict: Yes
 ├── Overcast:
    |└── Predict: Yes
 └── Rain:
    |Wind?
    | ├── Weak:
    |    |└── Predict: Yes
    | └── Strong:
    |    |└── Predict: No


Câu 2: Ứng dụng

In [48]:
# --- Tập dữ liệu 1: Vay vốn ---
loan_data = {
    'LichSuTinDung': ['Tốt', 'Xấu', 'Tốt', 'Trung bình', 'Tốt', 'Xấu', 'Trung bình', 'Tốt'],
    'ThuNhap': ['Cao', 'Cao', 'Thấp', 'Trung bình', 'Trung bình', 'Thấp', 'Cao', 'Trung bình'],
    'No': ['Thấp', 'Cao', 'Thấp', 'Thấp', 'Cao', 'Cao', 'Cao', 'Thấp'],
    'ChoVay': ['Có', 'Không', 'Có', 'Có', 'Không', 'Không', 'Có', 'Có']
}
df_loan = pd.DataFrame(loan_data)

# Chỉ định cột mục tiêu
loan_class_col = 'ChoVay'

# Xây dựng cây
loan_tree = build_decision_tree(df_loan, loan_class_col, visited=[])

# In cây ra màn hình
print("--- Cây Quyết Định: Dự Đoán Cho Vay ---")
print_tree(loan_tree)

LichSuTinDung Tốt
No Thấp
No Cao
LichSuTinDung Xấu
LichSuTinDung Trung bình
--- Cây Quyết Định: Dự Đoán Cho Vay ---
LichSuTinDung?
 ├── Tốt:
    |No?
    | ├── Thấp:
    |    |└── Predict: Có
    | └── Cao:
    |    |└── Predict: Không
 ├── Xấu:
    |└── Predict: Không
 └── Trung bình:
    |└── Predict: Có


In [ ]:
# --- Kiểm tra cây Vay vốn ---
print("\n--- Kiểm tra dự đoán cho các hồ sơ mới ---")

# Ví dụ 1: Khách hàng có lịch sử tín dụng tốt
example1 = {'LichSuTinDung': 'Tốt', 'ThuNhap': 'Cao', 'No': 'Thấp'}
print("Hồ sơ 1:", example1)
classify_example(example1, loan_tree)

# Ví dụ 2: Khách hàng có lịch sử tín dụng xấu nhưng thu nhập cao
example2 = {'LichSuTinDung': 'Xấu', 'ThuNhap': 'Cao', 'No': 'Thấp'}
print("\nHồ sơ 2:", example2)
classify_example(example2, loan_tree)

# Ví dụ 3: Khách hàng có lịch sử tín dụng trung bình và thu nhập cao
example3 = {'LichSuTinDung': 'Trung bình', 'ThuNhap': 'Cao', 'No': 'Cao'}
print("\nHồ sơ 3:", example3)
classify_example(example3, loan_tree)


--- Kiểm tra dự đoán cho các hồ sơ mới ---
Hồ sơ 1: {'LichSuTinDung': 'Tốt', 'ThuNhap': 'Cao', 'No': 'Thấp'}

Hồ sơ 2: {'LichSuTinDung': 'Xấu', 'ThuNhap': 'Cao', 'No': 'Thấp'}

Hồ sơ 3: {'LichSuTinDung': 'Trung bình', 'ThuNhap': 'Cao', 'No': 'Cao'}


'Có'